In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\text_Summerizer\\text_summarizer'

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir:Path
    STATUS_FILE: str
    ALL_REQUIRED_FILES: list

In [8]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories


In [11]:
class ConfriguationManager:
    def __init__(self,config_file_path=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self)-> DataValidationConfig:
        config=self.config.data_validation
        create_directories([config.root_dir])

        create_directories([config.root_dir])

        data_validation_config=DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            ALL_REQUIRED_FILES=config.ALL_REQUIRED_FILES,
        )

        return data_validation_config

    



In [13]:
import os
from textSummarizer.logging import logger


In [14]:
class DataValidation:
    def __init__(self,config:DataValidationConfig):
        self.config=config

    def validate_all_files_exists(self) ->bool:
        try:
            validation_status=None

            all_files=os.listdir(os.path.join("artifacts","data_ingestion","samsum_dataset"))
            for file in all_files:
                if file not in self.config.ALL_REQUIRED_FILES:
                    validation_status=False
                    with open(self.config.STATUS_FILE,"w") as f:
                        f.write(f"validation status {validation_status}")
                
                else:
                    validation_status=True
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"validation status: {validation_status}")


            return validation_status
        
        except Exception as e:
            raise e

In [16]:
try:
    config=ConfriguationManager()
    data_validation_config=config.get_data_validation_config()
    data_validation=DataValidation(config=data_validation_config)
    data_validation.validate_all_files_exists()
except Exception as e:
    raise e

[2025-10-09 19:05:58,581: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-09 19:05:58,592: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-09 19:05:58,593: INFO: common: created directory at: artifacts]
[2025-10-09 19:05:58,595: INFO: common: created directory at: artifacts/data_validation]
[2025-10-09 19:05:58,598: INFO: common: created directory at: artifacts/data_validation]


In [ ]:
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM

model_path="D:/text_Summerizer/text_summarizer/research/falcon_text_sum"
tokenizer_path="D:/text_Summerizer/text_summarizer/research/tokenizer_falcon"

model=AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer=AutoTokenizer.from_pretrained(tokenizer_path)

In [5]:
from transformers import pipeline

In [3]:
original_text="""Once upon a time in a small village nestled between green hills, there lived a young boy named Arjun. He was curious, brave, and loved exploring the forests near his home. One day, while wandering deeper than ever before, he stumbled upon an injured baby deer trapped under a fallen tree. Despite being scared, Arjun carefully lifted the branches and freed the animal.
Grateful, the deer followed him back to the village, and soon, the villagers noticed that it had extraordinary abilities—it could find hidden water sources, alert people to danger, and even heal minor wounds with its presence. Word of the magical deer spread, attracting travelers and merchants from faraway lands.
However, not everyone had good intentions. A greedy landowner wanted to capture the deer and exploit its powers for profit. Arjun, understanding the danger, devised a plan. He led the deer to the hills and created a secret sanctuary where only those with pure hearts could find it.
Years passed, and Arjun grew up to become a wise protector of the forest. The deer lived safely, helping those who respected nature. The village prospered, and people learned an important lesson: true courage and kindness can protect what is precious, and sometimes, the smallest acts of bravery can change the fate of many."""

In [4]:
gen_kwargs = {
    "max_length": 300,
    "min_length": 100,
    "num_beams": 4,
    "length_penalty": 2.0,  # encourages shorter summaries
    "early_stopping": True,
}


In [6]:
pipe = pipeline("summarization", model=model, tokenizer=tokenizer)

Device set to use cpu


In [7]:
text = "summarize: " + original_text
pipe(text, **gen_kwargs)

Your max_length is set to 300, but your input_length is only 293. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=146)
Both `max_new_tokens` (=256) and `max_length`(=300) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'summary_text': 'Arjun led the deer to the hills and created a secret sanctuary where only those with pure hearts could find it . The village prospered, and people learned an important lesson: true courage and kindness can protect what is precious, and sometimes, the smallest acts of bravery can change the fate of many . villagers noticed that it had extraordinary abilities—it could find hidden water sources, alert people to danger, and even heal minor wounds with its presence .'}]